In [1]:
import pandas as pd
import sys
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sentence_transformers import SentenceTransformer
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor

sys.path.append("../")
import src.data_utils as d_u
import src.feats_generation as f_g
import src.eval_utils as e_u

c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DungeonMaster3000\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!


Сначала с помощью подбора гиперпараметров посмотрим, какое наилучшее решение мы можем получить, используя линейную регрессию с L2 регуляризацией

In [2]:
# Получим предобработанный датасет

df = d_u.get_preprocess_data()

X = df.drop(columns=["commit_message", "bugs"])
y = df.bugs

msg_embs = f_g.pretrained_model_sentence_emb(df.commit_message.values)

X = np.concatenate((X.to_numpy(), msg_embs), axis=1)

In [3]:
# конфигурация для поиска наилучшего решения

ridge_config = {
    "alpha": [0.5, 1.0, 1.5, 5.0, 10],
    "solver": ["auto", "svd", "cholesky", "sag"],
    "random_state": [42],
    "tol": [1e-2, 1e-3, 1e-4]
}


In [14]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_ridge = Ridge(random_state=42)
gs_ridge = GridSearchCV(model_ridge, ridge_config, cv=cv, scoring="neg_mean_squared_error", verbose=1)
search = gs_ridge.fit(X, y)

# Посмотрим на лучшие параметры и лучшую оценку

print(gs_ridge.best_params_)
print(gs_ridge.best_score_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
{'alpha': 0.5, 'random_state': 42, 'solver': 'sag', 'tol': 0.01}
-1.3645594120400553


Коэффициент детерминации для линейной модели

In [20]:
model_ridge = Ridge(random_state=42)
gs_ridge = GridSearchCV(model_ridge, ridge_config, cv=cv, scoring="r2", verbose=1)
search = gs_ridge.fit(X, y)

# Посмотрим на лучшие параметры и лучшую оценку

print(gs_ridge.best_params_)
print(gs_ridge.best_score_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
{'alpha': 1.0, 'random_state': 42, 'solver': 'sag', 'tol': 0.001}
0.5903246804521681


Лучшее качество для линейной модели с L2 регуляризацией - 1.3645594120400553

Теперь подберем параметры для нейронной сети из sklearn

In [22]:
nn_config = {
    "hidden_layer_sizes": [(100,), (300, 100,), (300, 100, 50)],
    "activation": ["logistic", "tanh", "relu"],
    "solver": ["sgd", "adam"],
    "learning_rate": ["constant", "invscaling", "adaptive"],
    "random_state": [42],
    "max_iter": [10000]
}

In [23]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_nn = MLPRegressor(random_state=42)
gs_nn = GridSearchCV(model_nn, nn_config, cv=cv, scoring="neg_mean_squared_error", verbose=1)
search = gs_nn.fit(X, y)

# Посмотрим на лучшие параметры и лучшую оценку

print(search.best_params_)
print(search.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
{'activation': 'relu', 'hidden_layer_sizes': (300, 100), 'learning_rate': 'constant', 'max_iter': 10000, 'random_state': 42, 'solver': 'sgd'}
-1.0978673099828316


Коэффициент детерминации для нейронной сети

In [24]:
model_nn = MLPRegressor(random_state=42)
gs_nn = GridSearchCV(model_nn, nn_config, cv=cv, scoring="r2", verbose=1)
search = gs_nn.fit(X, y)

# Посмотрим на лучшие параметры и лучшую оценку

print(search.best_params_)
print(search.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
{'activation': 'relu', 'hidden_layer_sizes': (300, 100), 'learning_rate': 'adaptive', 'max_iter': 10000, 'random_state': 42, 'solver': 'sgd'}
0.663177781335307


Лучшее качество для нейронной сети - 1.0978673099828316

Теперь попробуем подобрать параметры для градиентного бустинга

In [25]:
gb_config = {
    "loss": ["squared_error", "absolute_error", "huber"],
    "learning_rate": [0.1, 0.5, 1.0],
    "n_estimators": [100, 150, 200],
    "criterion": ["squared_error"],
    "max_depth": [3, 5, 7],
    "random_state": [42]    
}

In [27]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_gb = GradientBoostingRegressor(random_state=42)
gs_gb = GridSearchCV(model_gb, gb_config, cv=cv, scoring="neg_mean_squared_error", verbose=1)
search = gs_gb.fit(X, y)

# Посмотрим на лучшие параметры и лучшую оценку

print(search.best_params_)
print(search.best_score_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
{'criterion': 'squared_error', 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 3, 'n_estimators': 150, 'random_state': 42}
-1.4590573214275597


Коэффициент детерминации для градиентного бустинга

In [28]:
model_gb = GradientBoostingRegressor(random_state=42)
gs_gb = GridSearchCV(model_gb, gb_config, cv=cv, scoring="r2", verbose=1)
search = gs_gb.fit(X, y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
{'criterion': 'squared_error', 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 3, 'n_estimators': 150, 'random_state': 42}
0.5730774817621374


In [29]:
# Посмотрим на лучшие параметры и лучшую оценку
print(search.best_params_)
print(search.best_score_)

{'criterion': 'squared_error', 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 3, 'n_estimators': 150, 'random_state': 42}
0.5730774817621374


Градиентный бустинг дает качество на даннной задаче значительно хуже нежели Ridge и MLP

Попробуем специализированные библиотеки для градиентного бустинга

In [30]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_lgb = LGBMRegressor(random_state=42)

print("MSE:")
print(np.mean(cross_val_score(model_lgb, X, y, cv=cv, scoring="neg_mean_squared_error")))
print("R2:")
print(np.mean(cross_val_score(model_lgb, X, y, cv=cv, scoring="r2")))

MSE:
-1.7115090825707544
R2:
0.4976710368532662


In [31]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_cgb = CatBoostRegressor(random_state=42, verbose=0)

print("MSE:")
print(np.mean(cross_val_score(model_cgb, X, y, cv=cv, scoring="neg_mean_squared_error")))
print("R2:")
print(np.mean(cross_val_score(model_cgb, X, y, cv=cv, scoring="r2")))

MSE:
-1.4879989582292803
R2:
0.5650289740256935


У базового регрессора из catboost качество получилось близкое к качеству модели градиентного бустинга из sklearn с параметрами, подобранными по сетке. Попробуем применить его без ручной предобработки категориальных признаков

In [32]:
df = pd.read_csv("../data/raw/АВСОФТ_тест_ML_приложение.csv")
df.drop(columns=["commit_hash"], inplace=True)

df["commit_date"] = pd.to_datetime(df.commit_date)

df = f_g.encode_work_days(df)
df = f_g.encode_work_hours(df)

if "commit_date" in df.columns:
    df.drop(columns=["commit_date"], inplace=True)

X = df.drop(columns=["commit_message", "bugs"])
y = df.bugs

msg_embs = f_g.pretrained_model_sentence_emb(df.commit_message.values)

X = pd.concat((X, pd.DataFrame(msg_embs)), axis=1)

In [33]:
X.sample(3)

,repository_name,commit_author,no_work_d,work_d,no_work_h,work_h,0,1,2,3,...,502,503,504,505,506,507,508,509,510,511
229,sensor,Victor,0,1,0,1,-0.016877,0.027794,0.049338,0.006484,...,0.038219,-0.019692,-0.002961,0.113271,0.001113,-0.034759,-0.046529,0.037617,-0.080065,-0.019196
109,conductor,Wendy,0,1,0,1,0.016724,0.067799,-0.023899,-0.049798,...,-0.009636,-0.063183,-0.002601,0.025828,-0.016108,-0.094530,-0.004286,0.068062,-0.022065,-0.052371
270,sensor,Victor,0,1,1,0,0.010032,0.045232,-0.018128,0.035715,...,0.034483,0.004908,0.025319,-0.036731,-0.037718,-0.067906,-0.025513,0.016430,-0.076118,0.006357


In [35]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_cgb = CatBoostRegressor(random_state=42, verbose=0, cat_features=["repository_name", "commit_author"])

In [36]:
print("MSE:")
mse_score = np.mean(cross_val_score(model_cgb, X, y, cv=cv, scoring="neg_mean_squared_error"))

MSE:
-1.609263768264578


In [38]:
print(mse_score)

-1.609263768264578


In [37]:
print("R2:")
r2_score = np.mean(cross_val_score(model_cgb, X, y, cv=cv, scoring="r2"))
print(r2_score)

R2:
0.531847915952463


Вряд ли подбор параметров позволит приблизиться к лидеру(нейронная сеть), однако в дальнейшем стоит поэксперементировать

### Выводы

Наилучшим подходом в данный момент себя показало кодирование имени репозитория, используя One-Hot Encoder, деление на 3 группы авторов комммитов по числу найденных ими ошибок и кодирование их также с помощью One-Hot Encoder. Была подтверждена гипотеза, что, учитывая смысл(хотя бы частичный) сообщения коммита, можно лучше предсказывать число найденных багов. Время коммита было преобразовано в пару полезных признаков: рабочее-нерабочее время, рабочий-выходной день недели. Я испытал как TfidfVectorizer, так и предобученную мультиязыковую модель на основе Transformer и CNN для получения эмбеддинга предложения. Второй вариант дал гораздо более сильное уменьшение средней квадратичной ошибки. Были испробованы TruncatedSVD и PCA, так как данных очень мало, а размер получаемого вектора признаков для объекта превышает общее количество объектов в имеющемся датасете. Однако, данные подходы не показали сколько-нибудь значительного улучшения качества работы модели.

Лучше всего себя на данный момент показала двухслойная нейронная сеть с функцийе активации Relu. Ее mse на кросс-валидации составило 1.0978673099828316, а коэффициент детерминации - 0.663177781335307

Дальнейшая работа:

1. Обучить двух(трех)слойную нейронную сеть с помощью фрэймворка Pytorch с использованием слоя Embeddings для категориальных переменных, таких как имя репозитория, имя автора коммита, рабочее время, будний день недели.

2. Попробовать применить стэкинг

3. Подумать над тем, какие еще признаки можно сгенерировать вручную на основе времени коммита и текста сообщения коммита.

4. Попробовать подобрать параметры для моделей из специализированных библиотек для градиентного бустинга(lightgbm и catboost)
